# HSI Classification: Local Similarity Analysis

This notebook demonstrates the hyperspectral image classification pipeline with focus on **local similarity metrics** for thesis research.

## Contents
1. Data Loading & Preprocessing
2. Patch Extraction
3. Similarity Metrics Exploration
4. Model Training with MLflow
5. Evaluation & Visualization

## 1. Setup & Imports

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Import our modules
from src.data import load_hsi, load_benchmark_dataset, preprocess_pipeline
from src.features import extract_patches, split_dataset
from src.models import (
    rbf_similarity, sam_similarity, local_similarity_matrix,
    SVMClassifier, create_model, train_svm, train_neural_network,
    compute_all_metrics, print_classification_report
)

# MLflow
import mlflow
mlflow.set_tracking_uri('mlruns')

print("✓ All imports successful!")

## 2. Generate Synthetic Data (for testing)

Replace this with actual HSI data loading in practice.

In [ ]:
# Generate synthetic hyperspectral cube for demonstration
np.random.seed(42)

# Dimensions
height, width, n_bands = 100, 100, 200
n_classes = 5

# Create class-specific spectral signatures
class_spectra = np.random.randn(n_classes, n_bands)

# Generate ground truth
ground_truth = np.zeros((height, width), dtype=np.int32)
for c in range(n_classes):
    mask = np.random.random((height, width)) < (1 / n_classes)
    ground_truth[mask] = c

# Generate HSI cube based on ground truth
hsi_cube = np.zeros((height, width, n_bands), dtype=np.float32)
for c in range(n_classes):
    mask = ground_truth == c
    hsi_cube[mask] = class_spectra[c] + 0.1 * np.random.randn(np.sum(mask), n_bands)

print(f"HSI Cube shape: {hsi_cube.shape}")
print(f"Ground Truth shape: {ground_truth.shape}")
print(f"Classes: {np.unique(ground_truth)}")

## 3. Preprocessing

In [ ]:
# Apply preprocessing pipeline
result = preprocess_pipeline(
    hsi_cube,
    remove_water=False,  # No water bands in synthetic data
    reduce_dims='pca',
    n_components=30,
    normalize='minmax',
    log_to_mlflow=False
)

processed_data = result['data']
print(f"Processed shape: {processed_data.shape}")

## 4. Patch Extraction

In [ ]:
# Extract patches
window_size = 5
dataset = extract_patches(
    processed_data,
    ground_truth,
    window_size=window_size,
    include_background=False,
    log_to_mlflow=False
)

print(f"Patches shape: {dataset.patches.shape}")
print(f"Number of samples: {dataset.n_samples}")

# Split into train/val/test
train_set, val_set, test_set = split_dataset(
    dataset,
    train_ratio=0.6,
    val_ratio=0.2,
    test_ratio=0.2,
    stratify=True,
    log_to_mlflow=False
)

print(f"Train: {train_set.n_samples}, Val: {val_set.n_samples}, Test: {test_set.n_samples}")

## 5. Local Similarity Analysis

This is the core of the thesis research.

In [ ]:
# Analyze local similarity for a sample patch
sample_patch = train_set.patches[0]

# Different similarity metrics
sigmas = [0.1, 0.5, 1.0, 2.0]

fig, axes = plt.subplots(2, len(sigmas), figsize=(16, 8))

# RBF similarity with different sigma values
for i, sigma in enumerate(sigmas):
    sim_matrix = local_similarity_matrix(sample_patch, metric='rbf', sigma=sigma)
    
    axes[0, i].imshow(sim_matrix, cmap='hot', vmin=0, vmax=1)
    axes[0, i].set_title(f'RBF (σ={sigma})')
    axes[0, i].axis('off')

# SAM vs Euclidean
sim_sam = local_similarity_matrix(sample_patch, metric='sam')
sim_euc = local_similarity_matrix(sample_patch, metric='euclidean')

axes[1, 0].imshow(sim_sam, cmap='hot', vmin=0, vmax=1)
axes[1, 0].set_title('SAM Similarity')
axes[1, 0].axis('off')

axes[1, 1].imshow(sim_euc, cmap='hot', vmin=0, vmax=1)
axes[1, 1].set_title('Euclidean Similarity')
axes[1, 1].axis('off')

# Hide unused axes
for j in range(2, len(sigmas)):
    axes[1, j].axis('off')

plt.suptitle('Local Similarity Analysis', fontsize=14)
plt.tight_layout()
plt.show()

## 6. SVM Classification

In [ ]:
# Prepare flattened features for SVM
X_train = train_set.get_center_pixels()
y_train = train_set.labels

X_val = val_set.get_center_pixels()
y_val = val_set.labels

X_test = test_set.get_center_pixels()
y_test = test_set.labels

print(f"X_train shape: {X_train.shape}")

# Train SVM with RBF kernel (relates to RBF similarity in thesis)
svm = train_svm(
    X_train, y_train,
    X_val, y_val,
    kernel='rbf',
    C=10.0,
    gamma='scale',
    experiment_name='hsi_classification',
    run_name='SVM_RBF_baseline',
    log_to_mlflow=True
)

# Evaluate on test set
y_pred = svm.predict(X_test)
print_classification_report(y_test, y_pred)

## 7. Neural Network Classification

In [ ]:
import torch
from src.models import create_model, train_neural_network, create_data_loaders, TrainingConfig

# Get dimensions
n_bands = train_set.patches.shape[-1]
n_classes_nn = len(np.unique(train_set.labels))

# Create 3D-CNN model
model = create_model(
    model_type='3d_cnn',
    n_bands=n_bands,
    n_classes=n_classes_nn,
    patch_size=window_size
)
print(model)

# Create data loaders
train_loader = create_data_loaders(train_set.patches, train_set.labels, batch_size=32, data_format='3d')
val_loader = create_data_loaders(val_set.patches, val_set.labels, batch_size=32, shuffle=False, data_format='3d')

# Training config
config = TrainingConfig(
    experiment_name='hsi_classification',
    run_name='3D_CNN_experiment',
    n_epochs=20,
    batch_size=32,
    learning_rate=0.001,
    patience=10
)

# Train (set log_to_mlflow=True to track with MLflow)
model, history = train_neural_network(
    model, train_loader, val_loader,
    config=config,
    similarity_config={'metric': 'patch_based', 'window_size': window_size},
    log_to_mlflow=True
)

## 8. Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Validation')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()

axes[1].plot(history['train_acc'], label='Train')
axes[1].plot(history['val_acc'], label='Validation')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

## 9. MLflow UI

Run the following command in terminal to view experiments:

```bash
cd hsi_classification
mlflow ui --port 5000
```

Then open http://localhost:5000 in your browser.

## 10. Run Streamlit Dashboard

```bash
cd hsi_classification
streamlit run src/visualization/visualize.py --server.port 8501
```

Then open http://localhost:8501 in your browser.